# Introduction
[Chainer](http://chainer.org/) とはニューラルネットの実装を簡単にしたフレームワークです。

* 今回は言語の分野でニューラルネットを適用してみました。

![](./pictures/Chainer.jpg)

* 今回は言語モデルを作成していただきます。


言語モデルとはある単語が来たときに次の単語に何が来やすいかを予測するものです。

言語モデルにはいくつか種類があるのでここでも紹介しておきます。

* n-グラム言語モデル
 * 単語の数を単純に数え挙げて作成されるモデル。考え方としてはデータにおけるある単語の頻度に近い
* ニューラル言語モデル
 * 単語の辞書ベクトルを潜在空間ベクトルに落とし込み、ニューラルネットで次の文字を学習させる手法

* リカレントニューラル言語モデル
 * 基本的なアルゴリズムはニューラル言語モデルと同一だが過去に使用した単語を入力に加えることによって文脈を考慮した言語モデルの学習が可能となる。ニューラル言語モデルとは異なり、より古い情報も取得可能

以下では、このChainerを利用しデータを準備するところから実際に言語モデルを構築し学習・評価を行うまでの手順を解説します。

1. [各種ライブラリ導入](#Loading-the-Library) 
2. [初期設定](#Initial-Setting) 
3. [データ入力](#Input-Data)
4. [リカレントニューラル言語モデル設定](#Reccurent-neural-language-model) 
5. [学習を始める前の設定](#Before-Setting)
6. [パラメータ更新方法（確率的勾配法）](#SGD)
7. [言語の予測](#Predict)

もしGPUを使用したい方は、以下にまとめてあるのでご参考ください。

[Chainer を用いてリカレントニューラル言語モデル作成のサンプルコードを解説してみた](http://qiita.com/GushiSnow/private/b34da4962dd930d1487a)



## Loading the Library

Chainerの言語処理では多数のライブラリを導入します。



In [214]:
import time
import math
import sys
import pickle
import copy
import os

import numpy as np
from chainer import cuda, Variable, FunctionSet, optimizers
import chainer.functions as F

`導入するライブラリの代表例は下記です。

* `numpy`: 行列計算などの複雑な計算を行なうライブラリ
* `chainer`: Chainerの導入


## Initial Setting

* 学習回数、ユニット数、確率的勾配法に使用するデータの数、学習に使用する文字列の長さ、勾配法で使用する敷居値、学習データの格納場所、モデルの出力場所を設定しています。



In [221]:
n_epochs    = 10
n_units     = 128
batchsize   = 50
bprop_len   = 50
grad_clip   = 5
data_dir = "data_hands_on"
checkpoint_dir = "cv"

## Input Data

学習用にダウンロードしたファイルをプログラムに読ませる処理を関数化しています

* 学習データをバイナリ形式で読み込んでいます。
* 文字データを確保するための行列を定義しています。
* データを単語をキー、長さを値とした辞書データにして行列データセットに登録しています。

学習データ、単語の長さ、語彙数を取得しています。
上記をそれぞれ行列データとして保持しています。


In [222]:
# input data
def load_data():
    vocab = {}
    print ('%s/linux_source.c'% data_dir)
    words = open('%s/linux_source.c' % data_dir, 'rb').read()
    words = list(words)
    dataset = np.ndarray((len(words),), dtype=np.int32)
    for i, word in enumerate(words):
        if word not in vocab:
            vocab[word] = len(vocab)
        dataset[i] = vocab[word]
    print('corpus length:', len(words))
    print('vocab size:', len(vocab))
    return dataset, words, vocab

if not os.path.exists(checkpoint_dir):
    os.mkdir(checkpoint_dir)
    
train_data, words, vocab = load_data()

## Reccurent neural language model

RNNLM(リカレントニューラル言語モデルの設定を行っています）

* EmbedIDで行列変換を行い、疎なベクトルを密なベクトルに変換しています。
* 出力が4倍の理由は入力層、出力層、忘却層、前回の出力をLSTMでは入力に使用するためです。
* 隠れ層に前回保持した隠れ層の状態を入力することによってLSTMを実現しています。
* ドロップアウトにより過学習するのを抑えています。
* 予測を行なうメソッドも実装しており、入力されたデータ、状態を元に次の文字列と状態を返すような関数になっています。
* モデルの初期化を行なう関数もここで定義しています。


In [223]:
class CharRNN(FunctionSet):

    def __init__(self, n_vocab, n_units):
        super(CharRNN, self).__init__(
            embed = F.EmbedID(n_vocab, n_units),
            l1_x = F.Linear(n_units, 4*n_units),
            l1_h = F.Linear(n_units, 4*n_units),
            l2_h = F.Linear(n_units, 4*n_units),
            l2_x = F.Linear(n_units, 4*n_units),
            l3   = F.Linear(n_units, n_vocab),
        )
        for param in self.parameters:
            param[:] = np.random.uniform(-0.08, 0.08, param.shape)

    def forward_one_step(self, x_data, y_data, state, train=True, dropout_ratio=0.5):
        x = Variable(x_data, volatile=not train)
        t = Variable(y_data, volatile=not train)

        h0      = self.embed(x)
        h1_in   = self.l1_x(F.dropout(h0, ratio=dropout_ratio, train=train)) + self.l1_h(state['h1'])
        c1, h1  = F.lstm(state['c1'], h1_in)
        h2_in   = self.l2_x(F.dropout(h1, ratio=dropout_ratio, train=train)) + self.l2_h(state['h2'])
        c2, h2  = F.lstm(state['c2'], h2_in)
        y       = self.l3(F.dropout(h2, ratio=dropout_ratio, train=train))
        state   = {'c1': c1, 'h1': h1, 'c2': c2, 'h2': h2}

        return state, F.softmax_cross_entropy(y, t)

    def predict(self, x_data, state):
        x = Variable(x_data, volatile=True)

        h0      = self.embed(x)
        h1_in   = self.l1_x(h0) + self.l1_h(state['h1'])
        c1, h1  = F.lstm(state['c1'], h1_in)
        h2_in   = self.l2_x(h1) + self.l2_h(state['h2'])
        c2, h2  = F.lstm(state['c2'], h2_in)
        y       = self.l3(h2)
        state   = {'c1': c1, 'h1': h1, 'c2': c2, 'h2': h2}

        return state, F.softmax(y)

def make_initial_state(n_units, batchsize=50, train=True):
    return {name: Variable(np.zeros((batchsize, n_units), dtype=np.float32),
            volatile=not train)
            for name in ('c1', 'h1', 'c2', 'h2')}

RNNLM(リカレントニューラル言語モデルの設定を行っています）

* 作成したリカレントニューラル言語モデルを導入しています。
* 最適化の手法はRMSpropを使用
http://qiita.com/skitaoka/items/e6afbe238cd69c899b2a
* 初期のパラメータを-0.1〜0.1の間で与えています。


In [224]:
# Prepare RNNLM model
model = CharRNN(len(vocab), n_units)

optimizer = optimizers.RMSprop(lr=2e-3, alpha=0.95, eps=1e-8)
optimizer.setup(model.collect_parameters())

print("*-----------------------------------model----------------------------------*")
print(dir(model))
print("*----------------------------------embed-----------------------------------*")
print(dir(model.embed))
print("*--------------------------------model l1_x--------------------------------*")
print(dir(model.l1_x.W))
print(model.l1_x.label)
print(model.l1_x.parameter_names)

## Before Setting

* 学習データのサイズを取得 
* ジャンプの幅を設定（順次学習しない）
* パープレキシティを0で初期化 
* 最初の時間情報を取得 
* 初期状態を現在の状態に付与 
* 状態の初期化 
* 損失を0で初期化

In [225]:
whole_len    = train_data.shape[0]
jump         = whole_len / batchsize
epoch        = 0
start_at     = time.time()
cur_at       = start_at
state        = make_initial_state(n_units, batchsize=batchsize)
accum_loss   = Variable(np.zeros((), dtype=np.float32))

## SGD

* 確率的勾配法を用いて学習している。
* 一定のデータを選択し損失計算をしながらパラメータ更新をしている。
* 逐次尤度の計算も行っている。

* 適宜学習データのパープレキシティも計算している

* バックプロパゲーションでパラメータを更新する。
* [truncate](http://kiyukuta.github.io/2013/12/09/mlac2013_day9_recurrent_neural_network_language_model.html#recurrent-neural-network)はどれだけ過去の履歴を見るかを表している。
* optimizer.clip_gradsの部分でL2正則化をかけている。
* 過学習を抑えるために学習効率を徐々に下げている。

In [226]:
for i in range(int(jump * n_epochs)):
    x_batch = np.array([train_data[(jump * j + i) % whole_len]
                        for j in range(batchsize)])
    y_batch = np.array([train_data[(jump * j + i + 1) % whole_len]
                        for j in range(batchsize)])

    state, loss_i = model.forward_one_step(x_batch, y_batch, state, dropout_ratio=0.5)
    accum_loss   += loss_i

    if (i + 1) % bprop_len == 0:  # Run truncated BPTT
        now = time.time()
        print('{}/{}, train_loss = {}, time = {:.2f}'.format((i+1)/bprop_len, jump, accum_loss.data / bprop_len, now-cur_at))
        cur_at = now

        optimizer.zero_grads()
        accum_loss.backward()
        accum_loss.unchain_backward()  # truncate
        accum_loss = Variable(np.zeros((), dtype=np.float32))

        optimizer.clip_grads(grad_clip)
        optimizer.update()

    if (i + 1) % 10000 == 0:    
        fn = ('%s/charrnn_epoch_%.2f.chainermodel' % (checkpoint_dir, float(i)/jump))
        pickle.dump(copy.deepcopy(model).to_cpu(), open(fn, 'wb'))

    if (i + 1) % jump == 0:
        epoch += 1

        if epoch >= 10:
            optimizer.lr *= 0.97
            print('decayed learning rate by a factor {} to {}'.format(0.97, optimizer.lr))

    sys.stdout.flush()


## Predict

* 学習したデータを再度入力
* 入力データを辞書として保持


In [227]:
np.random.seed(123)

# load vocabulary
vocab = {}
def load_predict_data(filename):
    global vocab, n_vocab
    #words = open(filename).read().replace('\n', '<eos>').strip().split()
    words = open(filename).read().strip().split()
    dataset = np.ndarray((len(words),), dtype=np.int32)
    for i, word in enumerate(words):
        if word not in vocab:
            vocab[word] = len(vocab)
        dataset[i] = vocab[word]
    return dataset

train_data = load_predict_data('data_hands_on/linux_source.c')

ivocab = {}
ivocab = {v:k for k, v in vocab.items()}



* 学習したモデルを取得
* モデルからユニット数を取得
* 最初の空文字を設定

In [230]:
# load model
model = pickle.load(open("cv/charrnn_epoch_9.45.chainermodel", 'rb'))
n_units = model.embed.W.shape[1]

# initialize generator
state = make_initial_state(n_units, batchsize=1, train=False)

prev_char = np.array([0], dtype=np.int32)

* 学習したモデルを利用して文字の予測を行なう。
* 予測で出力された文字と状態を次の入力に使用する。

In [231]:
for i in range(1000):
    state, prob = model.predict(prev_char, state)

    index = np.argmax(cuda.to_cpu(prob.data))
    sys.stdout.write(ivocab[index] + " ")

    prev_char = np.array([index], dtype=np.int32)

print